In [13]:
import requests
import sqlite3

def fetch_artists_from_deezer(limit=10):
    url = f"https://api.deezer.com/chart/0/artists?limit={limit}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['data']
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

def create_db():
    conn = sqlite3.connect("artists.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS artists (
            id INTEGER PRIMARY KEY,
            name TEXT,
            link TEXT,
            picture TEXT,
            picture_small TEXT,
            picture_medium TEXT,
            picture_big TEXT,
            picture_xl TEXT,
            radio BOOLEAN,
            tracklist TEXT,
            position INTEGER,
            type TEXT
        )
    ''')
    conn.commit()
    conn.close()

def save_artists_to_db(artists):
    conn = sqlite3.connect("artists.db")
    cursor = conn.cursor()

    for artist in artists:
        cursor.execute('''
            INSERT OR REPLACE INTO artists (
                id, name, link, picture, picture_small, picture_medium,
                picture_big, picture_xl, radio, tracklist, position, type
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            artist['id'],
            artist['name'],
            artist['link'],
            artist['picture'],
            artist['picture_small'],
            artist['picture_medium'],
            artist['picture_big'],
            artist['picture_xl'],
            artist['radio'],
            artist['tracklist'],
            artist['position'],
            artist['type']
        ))

    conn.commit()
    conn.close()

# 🧪 Виконання:
if __name__ == "__main__":
    create_db()
    artists = fetch_artists_from_deezer(limit=50)
    save_artists_to_db(artists)
    print(f"✅ {len(artists)} artists saved to database.")


✅ 50 artists saved to database.


In [14]:
import requests
import sqlite3
import time

def create_tracks_table():
    conn = sqlite3.connect("artists.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tracks (
            id INTEGER PRIMARY KEY,
            title TEXT,
            link TEXT,
            duration INTEGER,
            preview TEXT,
            position INTEGER,
            rank INTEGER,
            explicit_lyrics BOOLEAN,
            artist_id INTEGER,
            artist_name TEXT,
            album_id INTEGER,
            album_title TEXT,
            album_cover TEXT
        )
    ''')
    conn.commit()
    conn.close()

def fetch_and_save_tracks():
    conn = sqlite3.connect("artists.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name, tracklist FROM artists")
    artists = cursor.fetchall()
    
    for artist_id, artist_name, tracklist_url in artists:
        try:
            print(f"Fetching tracks for {artist_name}...")
            response = requests.get(tracklist_url)
            if response.status_code == 200:
                tracks = response.json()['data']
                for track in tracks:
                    # Basic album info
                    album = track.get("album", {})
                    cursor.execute('''
                        INSERT OR REPLACE INTO tracks (
                            id, title, link, duration, preview, position, rank,
                            explicit_lyrics, artist_id, artist_name,
                            album_id, album_title, album_cover
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        track['id'],
                        track['title'],
                        track['link'],
                        track['duration'],
                        track['preview'],
                        track.get('position'),
                        track['rank'],
                        track['explicit_lyrics'],
                        artist_id,
                        artist_name,
                        album.get('id'),
                        album.get('title'),
                        album.get('cover_medium')
                    ))
                conn.commit()
            else:
                print(f"Error fetching tracks for {artist_name} ({response.status_code})")
        except Exception as e:
            print(f"Error for {artist_name}: {e}")
        time.sleep(0.3)  # avoid rate-limiting
    
    conn.close()

# 🧪 Виконання:
if __name__ == "__main__":
    create_tracks_table()
    fetch_and_save_tracks()
    print("✅ Tracks saved to database.")


Fetching tracks for Eminem...
Fetching tracks for Linkin Park...
Fetching tracks for AC/DC...
Fetching tracks for Metallica...
Fetching tracks for Kanye West...
Fetching tracks for Queen...
Fetching tracks for Tiësto...
Fetching tracks for System of a Down...
Fetching tracks for Rammstein...
Fetching tracks for David Guetta...
Fetching tracks for OneRepublic...
Fetching tracks for Justin Bieber...
Fetching tracks for Ed Sheeran...
Fetching tracks for Imagine Dragons...
Fetching tracks for Twenty One Pilots...
Fetching tracks for Okean Elzy...
Fetching tracks for Loboda...
Fetching tracks for Lana Del Rey...
Fetching tracks for The Hardkiss...
Fetching tracks for Ariana Grande...
Fetching tracks for The Weeknd...
Fetching tracks for Баста...
Fetching tracks for Король и Шут...
Fetching tracks for Artik & Asti...
Fetching tracks for Бумбокс...
Fetching tracks for Сплин...
Fetching tracks for Кино...
Fetching tracks for Би-2...
Fetching tracks for Minelli...
Fetching tracks for Земфира...